<a href="https://colab.research.google.com/github/gachet/ad-1-24/blob/main/pca/ejem-pres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # importando numpy
from scipy import stats # importando scipy.stats
import pandas as pd # importando pandas



In [26]:
x = [19,39,30,30,15,15,15,30]
y= [63,74,87,23,35,43,32,73]
datos = np.stack((x, y), axis=0)
datos

array([[19, 39, 30, 30, 15, 15, 15, 30],
       [63, 74, 87, 23, 35, 43, 32, 73]])

In [28]:
A=np.cov(datos)
A

array([[ 85.83928571, 121.32142857],
       [121.32142857, 551.07142857]])

In [31]:
x1 = [19.,39.,30.,30.,15.,15.,15.,30.]
x2 =  [63.,74.,87.,23.,35.,43.,32.,73.]
N=np.size(x1)
X = np.column_stack([x1, x2])
X -= X.mean(axis=0)
fact = N
by_hand = np.dot(X.T, X.conj()) / fact
print(by_hand)

[[ 75.109375 106.15625 ]
 [106.15625  482.1875  ]]


In [37]:
autovalor, autovector = np.linalg.eig(by_hand )
autovalor

array([ 49.08951396, 508.20736104])

In [38]:
autovector

array([[-0.97124992, -0.23806218],
       [ 0.23806218, -0.97124992]])

In [ ]:
autovalor, autovector = np.linalg.eig(A)

In [ ]:
autovalor

array([ 56.10230167, 580.80841262])

In [ ]:
autovector

array([[-0.97124992, -0.23806218],
       [ 0.23806218, -0.97124992]])

In [17]:
x = [2.5,0.5,2.2,1.9,3.1,2.3,2,1,1.5,1.2]
y= [2.4,0.7,2.9,2.2,3.0,2.7,1.6,1.1,1.6,0.9]
datos = np.stack((x, y), axis=0)
datos

array([[2.5, 0.5, 2.2, 1.9, 3.1, 2.3, 2. , 1. , 1.5, 1.2],
       [2.4, 0.7, 2.9, 2.2, 3. , 2.7, 1.6, 1.1, 1.6, 0.9]])

In [16]:
b1 = [2.5,0.5,2.2,1.9,3.1,2.3,2,1,1.5,1.2]
b2 =  [2.4,0.7,2.9,2.2,3.0,2.7,1.6,1.1,1.6,0.9]
N=np.size(b1)
X = np.column_stack([b1, b2])
X -= X.mean(axis=0)
fact = N-1
by_hand = np.dot(X.T, X.conj()) / fact
print(by_hand)

[[0.60177778 0.60422222]
 [0.60422222 0.71655556]]


In [18]:
Z = np.cov(x,y)
Z

array([[0.60177778, 0.60422222],
       [0.60422222, 0.71655556]])

In [19]:
A=np.cov(datos)
A

array([[0.60177778, 0.60422222],
       [0.60422222, 0.71655556]])

In [6]:
autovalor, autovector = np.linalg.eig(A)

In [7]:
autovalor

array([0.04702256, 1.13951381])

In [8]:
autovector

array([[-0.73977066, -0.67285911],
       [ 0.67285911, -0.73977066]])